In [1]:
import pyspark
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql import Window

In [2]:
print(pyspark.__version__)

3.5.5


In [3]:
spark = SparkSession.builder \
    .appName("hotel_bookings") \
    .master("local[*]") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/19 18:14:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
hotels_df = spark.read.csv("hotels.csv", header=True, inferSchema=True)
guests_df = spark.read.csv("guests.csv", header=True, inferSchema=True)
bookings_df = spark.read.csv("bookings.csv", header=True, inferSchema=True)

In [5]:
hotels_df.createOrReplaceTempView("hotels")
guests_df.createOrReplaceTempView("guests")
bookings_df.createOrReplaceTempView("bookings")

In [6]:
hotels_df.printSchema()

root
 |-- hotel_id: integer (nullable = true)
 |-- hotel_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- star_rating: integer (nullable = true)



In [7]:
guests_df.printSchema()

root
 |-- guest_id: integer (nullable = true)
 |-- guest_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- country: string (nullable = true)



In [8]:
bookings_df.printSchema()

root
 |-- booking_id: integer (nullable = true)
 |-- guest_id: integer (nullable = true)
 |-- hotel_id: integer (nullable = true)
 |-- check_in_date: date (nullable = true)
 |-- check_out_date: date (nullable = true)
 |-- num_nights: integer (nullable = true)
 |-- room_rate: integer (nullable = true)



### Display the total number of bookings per hotel.

In [9]:
spark.sql(
    """
    SELECT
        b.hotel_id,
        h.hotel_name,
        COUNT(DISTINCT b.booking_id) AS booking_count
    FROM bookings b
    LEFT JOIN hotels h
    USING (hotel_id)
    GROUP BY 1, 2 
    ORDER BY 3 DESC
    """
).show()

+--------+--------------------+-------------+
|hotel_id|          hotel_name|booking_count|
+--------+--------------------+-------------+
|       4|         Huffman Inc|           14|
|       8|Davis, Crane and ...|           13|
|       5|Salazar, Porter a...|           13|
|       6|      Clark and Sons|           11|
|       7|          Chavez PLC|            9|
|      10|           Lopez Ltd|            8|
|       3|        Roach-Jarvis|            6|
|       2|         Smith Group|            6|
|       9|        Knight Group|            5|
|       1|        Lin-Williams|            5|
+--------+--------------------+-------------+



In [10]:
joined = bookings_df.join(
    hotels_df,
    on="hotel_id",
    how="left"
)

joined.groupBy("hotel_id", "hotel_name") \
    .agg(F.countDistinct(F.col("booking_id")).alias("booking_count")) \
    .orderBy(F.col("booking_count").desc()) \
    .show()

+--------+--------------------+-------------+
|hotel_id|          hotel_name|booking_count|
+--------+--------------------+-------------+
|       4|         Huffman Inc|           14|
|       8|Davis, Crane and ...|           13|
|       5|Salazar, Porter a...|           13|
|       6|      Clark and Sons|           11|
|       7|          Chavez PLC|            9|
|      10|           Lopez Ltd|            8|
|       3|        Roach-Jarvis|            6|
|       2|         Smith Group|            6|
|       9|        Knight Group|            5|
|       1|        Lin-Williams|            5|
+--------+--------------------+-------------+



### Find the average room rate charged by each hotel.

In [11]:
spark.sql(
    """
    SELECT
        b.hotel_id,
        h.hotel_name,
        ROUND(AVG(b.room_rate), 2) AS avg_rate
    FROM bookings b
    LEFT JOIN hotels h
    USING (hotel_id)
    GROUP BY 1, 2
    ORDER BY 3 DESC
    """
).show()

+--------+--------------------+--------+
|hotel_id|          hotel_name|avg_rate|
+--------+--------------------+--------+
|       1|        Lin-Williams|   220.0|
|       9|        Knight Group|   190.0|
|       5|Salazar, Porter a...|  184.62|
|       3|        Roach-Jarvis|  183.33|
|       6|      Clark and Sons|  181.82|
|       7|          Chavez PLC|  161.11|
|       4|         Huffman Inc|  157.14|
|      10|           Lopez Ltd|  156.25|
|       8|Davis, Crane and ...|  142.31|
|       2|         Smith Group|  133.33|
+--------+--------------------+--------+



In [12]:
joined_df = bookings_df.join(
    hotels_df,
    on="hotel_id",
    how="left"
)

joined_df.groupBy("hotel_id", "hotel_name") \
    .agg(F.round(F.avg(F.col("room_rate")), 2).alias("avg_rate")) \
    .orderBy(F.col("avg_rate").desc()) \
    .show()

+--------+--------------------+--------+
|hotel_id|          hotel_name|avg_rate|
+--------+--------------------+--------+
|       1|        Lin-Williams|   220.0|
|       9|        Knight Group|   190.0|
|       5|Salazar, Porter a...|  184.62|
|       3|        Roach-Jarvis|  183.33|
|       6|      Clark and Sons|  181.82|
|       7|          Chavez PLC|  161.11|
|       4|         Huffman Inc|  157.14|
|      10|           Lopez Ltd|  156.25|
|       8|Davis, Crane and ...|  142.31|
|       2|         Smith Group|  133.33|
+--------+--------------------+--------+



### List all guests who have stayed more than once.

In [13]:
spark.sql(
    """
    SELECT
        b.guest_id,
        g.guest_name,
        COUNT(b.guest_id) AS booking_count
    FROM bookings b
    LEFT JOIN guests g
    USING (guest_id)
    GROUP BY 1, 2
    HAVING COUNT(b.guest_id) > 1
    ORDER BY 3
    """
).show()

+--------+------------------+-------------+
|guest_id|        guest_name|booking_count|
+--------+------------------+-------------+
|      32|       Dylan Brady|            2|
|       8|   Jennifer Grimes|            2|
|      22|      Mary Jimenez|            2|
|      12| Christopher Smith|            2|
|      14|      Dennis Allen|            2|
|      29|Derrick Ramirez MD|            2|
|       1|     Sandra Turner|            2|
|      39|        Jesse Hill|            3|
|      13|Christopher Murphy|            3|
|      28|      Donna Osborn|            3|
|      26|     Jessica Perry|            3|
|       9|     Brett Shields|            3|
|       7|      Diane Howard|            3|
|      37|   David Hernandez|            3|
|       4|  Ashley Carpenter|            3|
|      31|      Scott Rhodes|            3|
|       5| Lindsay Blake DDS|            3|
|      23|    Mary Alexander|            3|
|      18|       Sandra Wall|            4|
|      30|   Adam Pennington|   

In [14]:
joined_df = bookings_df.join(
    guests_df,
    on="guest_id",
    how="left"
)

joined_df.groupBy("guest_id", "guest_name") \
    .agg(F.count(F.col("guest_id")).alias("booking_count")) \
    .filter(F.col("booking_count") > 1) \
    .orderBy(F.col("booking_count")) \
    .show()

+--------+------------------+-------------+
|guest_id|        guest_name|booking_count|
+--------+------------------+-------------+
|      32|       Dylan Brady|            2|
|       8|   Jennifer Grimes|            2|
|      22|      Mary Jimenez|            2|
|      12| Christopher Smith|            2|
|      14|      Dennis Allen|            2|
|      29|Derrick Ramirez MD|            2|
|       1|     Sandra Turner|            2|
|      39|        Jesse Hill|            3|
|      13|Christopher Murphy|            3|
|      28|      Donna Osborn|            3|
|      26|     Jessica Perry|            3|
|       9|     Brett Shields|            3|
|       7|      Diane Howard|            3|
|      37|   David Hernandez|            3|
|       4|  Ashley Carpenter|            3|
|      31|      Scott Rhodes|            3|
|       5| Lindsay Blake DDS|            3|
|      23|    Mary Alexander|            3|
|      18|       Sandra Wall|            4|
|      30|   Adam Pennington|   

### Which hotel had the longest average stay duration?

In [15]:
spark.sql(
    """
    SELECT
        b.hotel_id,
        h.hotel_name,
        ROUND(AVG(DATEDIFF(b.check_out_date, b.check_in_date)), 2) AS avg_stay
    FROM bookings b
    LEFT JOIN hotels h
    USING (hotel_id)
    GROUP BY 1, 2
    ORDER BY 3 DESC
    """
).show()

+--------+--------------------+--------+
|hotel_id|          hotel_name|avg_stay|
+--------+--------------------+--------+
|       3|        Roach-Jarvis|    7.33|
|      10|           Lopez Ltd|     7.0|
|       1|        Lin-Williams|     7.0|
|       7|          Chavez PLC|    6.44|
|       9|        Knight Group|     6.4|
|       5|Salazar, Porter a...|     6.0|
|       8|Davis, Crane and ...|    5.85|
|       4|         Huffman Inc|    4.71|
|       6|      Clark and Sons|    4.55|
|       2|         Smith Group|    4.17|
+--------+--------------------+--------+



In [16]:
joined_df = bookings_df.join(
    hotels_df,
    on="hotel_id",
    how="left"
)

joined_df.groupBy("hotel_id", "hotel_name") \
    .agg(F.round(F.avg(F.date_diff(F.col("check_out_date"), F.col("check_in_date"))), 2).alias("avg_stay")) \
    .orderBy(F.col("avg_stay").desc()) \
    .show()

+--------+--------------------+--------+
|hotel_id|          hotel_name|avg_stay|
+--------+--------------------+--------+
|       3|        Roach-Jarvis|    7.33|
|      10|           Lopez Ltd|     7.0|
|       1|        Lin-Williams|     7.0|
|       7|          Chavez PLC|    6.44|
|       9|        Knight Group|     6.4|
|       5|Salazar, Porter a...|     6.0|
|       8|Davis, Crane and ...|    5.85|
|       4|         Huffman Inc|    4.71|
|       6|      Clark and Sons|    4.55|
|       2|         Smith Group|    4.17|
+--------+--------------------+--------+



### List the total revenue generated by each hotel.

In [17]:
spark.sql(
    """
    SELECT
        b.hotel_id,
        h.hotel_name,
        SUM(b.room_rate * b.num_nights) AS total_revenue
    FROM bookings b
    LEFT JOIN hotels h
    USING (hotel_id)
    GROUP BY 1, 2
    ORDER BY 3 DESC
    """
).show()

+--------+--------------------+-------------+
|hotel_id|          hotel_name|total_revenue|
+--------+--------------------+-------------+
|       5|Salazar, Porter a...|        14800|
|       8|Davis, Crane and ...|        10300|
|       4|         Huffman Inc|         9750|
|       6|      Clark and Sons|         8950|
|       7|          Chavez PLC|         8800|
|       3|        Roach-Jarvis|         8350|
|      10|           Lopez Ltd|         7950|
|       1|        Lin-Williams|         7650|
|       9|        Knight Group|         5050|
|       2|         Smith Group|         3250|
+--------+--------------------+-------------+



In [18]:
bookings_hotels_joined_df = bookings_df.join(
    hotels_df,
    on="hotel_id",
    how="left"
)

bookings_hotels_joined_df.groupBy("hotel_id", "hotel_name") \
    .agg(F.sum(F.col("num_nights") * F.col("room_rate")).alias("total_revenue")) \
    .orderBy(F.col("total_revenue").desc()) \
    .show()

+--------+--------------------+-------------+
|hotel_id|          hotel_name|total_revenue|
+--------+--------------------+-------------+
|       5|Salazar, Porter a...|        14800|
|       8|Davis, Crane and ...|        10300|
|       4|         Huffman Inc|         9750|
|       6|      Clark and Sons|         8950|
|       7|          Chavez PLC|         8800|
|       3|        Roach-Jarvis|         8350|
|      10|           Lopez Ltd|         7950|
|       1|        Lin-Williams|         7650|
|       9|        Knight Group|         5050|
|       2|         Smith Group|         3250|
+--------+--------------------+-------------+



### List guests who booked rooms in 5-star hotels only.

In [19]:
guests_df.printSchema()

root
 |-- guest_id: integer (nullable = true)
 |-- guest_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- country: string (nullable = true)



In [20]:
bookings_df.printSchema()

root
 |-- booking_id: integer (nullable = true)
 |-- guest_id: integer (nullable = true)
 |-- hotel_id: integer (nullable = true)
 |-- check_in_date: date (nullable = true)
 |-- check_out_date: date (nullable = true)
 |-- num_nights: integer (nullable = true)
 |-- room_rate: integer (nullable = true)



In [21]:
hotels_df.printSchema()

root
 |-- hotel_id: integer (nullable = true)
 |-- hotel_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- star_rating: integer (nullable = true)



In [22]:
spark.sql(
    """
    SELECT
        g.guest_id,
        g.guest_name
    FROM guests g
    JOIN bookings b
    USING (guest_id)
    JOIN hotels h
    USING (hotel_id)
    GROUP BY 1, 2
    HAVING MIN(h.star_rating) = 5 AND MAX(h.star_rating) = 5
    """
).show()

+--------+---------------+
|guest_id|     guest_name|
+--------+---------------+
|      34|Travis Davidson|
|      14|   Dennis Allen|
|      21|    Emily Moore|
+--------+---------------+



In [23]:
all_joined_df = guests_df.join(
    bookings_df,
    on="guest_id",
    how="inner"
).join(
    hotels_df,
    on="hotel_id",
    how="inner"
)

grouped_df = all_joined_df.groupBy("guest_id", "guest_name") \
    .agg(
        F.min(F.col("star_rating")).alias("min_star"),
        F.max(F.col("star_rating")).alias("max_star")
    )

grouped_df.filter(
    (F.col("min_star") == 5)
    &
    (F.col("max_star") == 5)
).select(
    "guest_id",
    "guest_name"
).show()

+--------+---------------+
|guest_id|     guest_name|
+--------+---------------+
|      34|Travis Davidson|
|      14|   Dennis Allen|
|      21|    Emily Moore|
+--------+---------------+



### Find hotels that were booked by guests from more than 3 different countries.

In [24]:
spark.sql(
    """
    SELECT
        b.hotel_id,
        h.hotel_name,
        COUNT(DISTINCT g.country) AS unique_countries
    FROM bookings b
    LEFT JOIN guests g
    USING (guest_id)
    LEFT JOIN hotels h
    USING (hotel_id)
    GROUP BY 1, 2
    HAVING COUNT(DISTINCT g.country) > 3
    ORDER BY 3 DESC
    """
).show()

+--------+--------------------+----------------+
|hotel_id|          hotel_name|unique_countries|
+--------+--------------------+----------------+
|       4|         Huffman Inc|              12|
|       8|Davis, Crane and ...|              11|
|       5|Salazar, Porter a...|              11|
|       6|      Clark and Sons|               9|
|      10|           Lopez Ltd|               6|
|       7|          Chavez PLC|               6|
|       3|        Roach-Jarvis|               6|
|       2|         Smith Group|               6|
|       9|        Knight Group|               5|
|       1|        Lin-Williams|               5|
+--------+--------------------+----------------+



In [25]:
all_joined_df = bookings_df.join(
    guests_df,
    on="guest_id",
    how="left"
).join(
    hotels_df,
    on="hotel_id",
    how="left"
)

grouped_df = all_joined_df.groupBy("hotel_id", "hotel_name") \
    .agg(F.countDistinct(F.col("country")).alias("unique_country_count")) \
    .orderBy(F.col("unique_country_count").desc())

grouped_df.filter(F.col("unique_country_count") > 3) \
    .select("hotel_id", "hotel_name", "unique_country_count") \
    .show()

+--------+--------------------+--------------------+
|hotel_id|          hotel_name|unique_country_count|
+--------+--------------------+--------------------+
|       4|         Huffman Inc|                  12|
|       8|Davis, Crane and ...|                  11|
|       5|Salazar, Porter a...|                  11|
|       6|      Clark and Sons|                   9|
|      10|           Lopez Ltd|                   6|
|       7|          Chavez PLC|                   6|
|       3|        Roach-Jarvis|                   6|
|       2|         Smith Group|                   6|
|       9|        Knight Group|                   5|
|       1|        Lin-Williams|                   5|
+--------+--------------------+--------------------+



### Show top 5 guests who spent the most across all bookings.

In [26]:
spark.sql(
    """
    SELECT
        b.guest_id,
        g.guest_name,
        SUM(b.num_nights * b.room_rate) AS total_spent
    FROM bookings b
    LEFT JOIN guests g
    USING (guest_id)
    GROUP BY 1, 2
    ORDER BY 3 DESC
    LIMIT 5
    """
).show()

+--------+------------------+-----------+
|guest_id|        guest_name|total_spent|
+--------+------------------+-----------+
|      38|Mark Robertson Jr.|       7700|
|      40|      Andrew Smith|       5650|
|      26|     Jessica Perry|       4800|
|      25|      Juan Rosario|       4050|
|       5| Lindsay Blake DDS|       3400|
+--------+------------------+-----------+



In [27]:
joined_df = bookings_df.join(
    guests_df,
    on="guest_id",
    how="left"
)

joined_df.groupBy("guest_id", "guest_name") \
    .agg(F.sum(F.col("num_nights") * F.col("room_rate")).alias("total_spent")) \
    .orderBy(F.col("total_spent").desc()) \
    .limit(5) \
    .show()

+--------+------------------+-----------+
|guest_id|        guest_name|total_spent|
+--------+------------------+-----------+
|      38|Mark Robertson Jr.|       7700|
|      40|      Andrew Smith|       5650|
|      26|     Jessica Perry|       4800|
|      25|      Juan Rosario|       4050|
|       5| Lindsay Blake DDS|       3400|
+--------+------------------+-----------+



### Join guests and bookings and show guest names, their countries, and total nights stayed.

In [29]:
spark.sql(
    """
    SELECT
        b.guest_id,
        g.guest_name,
        g.country,
        SUM(b.num_nights) as total_nights
    FROM bookings b
    LEFT JOIN guests g
    USING (guest_id)
    GROUP BY 1, 2, 3
    ORDER BY 4 DESC
    """
).show()

+--------+------------------+--------------------+------------+
|guest_id|        guest_name|             country|total_nights|
+--------+------------------+--------------------+------------+
|      38|Mark Robertson Jr.|         Switzerland|          45|
|      25|      Juan Rosario|             Comoros|          38|
|      40|      Andrew Smith|                Chad|          25|
|      23|    Mary Alexander|French Southern T...|          23|
|      26|     Jessica Perry|          San Marino|          22|
|       5| Lindsay Blake DDS|            Paraguay|          22|
|      16|      Daniel Wells|Saint Kitts and N...|          21|
|      18|       Sandra Wall|             Albania|          21|
|      13|Christopher Murphy|              Norway|          21|
|       4|  Ashley Carpenter|         Puerto Rico|          21|
|      39|        Jesse Hill|Svalbard & Jan Ma...|          18|
|       8|   Jennifer Grimes|       Faroe Islands|          18|
|      31|      Scott Rhodes|           

In [33]:
joined_df = bookings_df.join(
    guests_df,
    on="guest_id",
    how="left"
)

joined_df.groupBy("guest_id", "guest_name", "country") \
    .agg(F.sum(F.col("num_nights")).alias("total_nights")) \
    .orderBy(F.col("total_nights").desc()) \
    .show()

+--------+------------------+--------------------+------------+
|guest_id|        guest_name|             country|total_nights|
+--------+------------------+--------------------+------------+
|      38|Mark Robertson Jr.|         Switzerland|          45|
|      25|      Juan Rosario|             Comoros|          38|
|      40|      Andrew Smith|                Chad|          25|
|      23|    Mary Alexander|French Southern T...|          23|
|      26|     Jessica Perry|          San Marino|          22|
|       5| Lindsay Blake DDS|            Paraguay|          22|
|      16|      Daniel Wells|Saint Kitts and N...|          21|
|      18|       Sandra Wall|             Albania|          21|
|      13|Christopher Murphy|              Norway|          21|
|       4|  Ashley Carpenter|         Puerto Rico|          21|
|      39|        Jesse Hill|Svalbard & Jan Ma...|          18|
|       8|   Jennifer Grimes|       Faroe Islands|          18|
|      31|      Scott Rhodes|           

### Add a new column to bookings: total_amount = num_nights * room_rate. Show top 5 highest value bookings.

In [34]:
spark.sql(
    """
    SELECT
        booking_id,
        num_nights * room_rate AS total_amount
    FROM bookings
    """
).show()

+----------+------------+
|booking_id|total_amount|
+----------+------------+
|         1|        1400|
|         2|        1800|
|         3|         600|
|         4|        1200|
|         5|         400|
|         6|         450|
|         7|        1000|
|         8|         600|
|         9|         600|
|        10|        1800|
|        11|         150|
|        12|         300|
|        13|         600|
|        14|        1800|
|        15|         450|
|        16|         700|
|        17|        1200|
|        18|        1350|
|        19|         150|
|        20|        1800|
+----------+------------+
only showing top 20 rows



In [36]:
bookings_df.withColumn(
    "total_amount",
    F.col("num_nights") * F.col("room_rate")
).select(
    "booking_id",
    "total_amount"
).show()

+----------+------------+
|booking_id|total_amount|
+----------+------------+
|         1|        1400|
|         2|        1800|
|         3|         600|
|         4|        1200|
|         5|         400|
|         6|         450|
|         7|        1000|
|         8|         600|
|         9|         600|
|        10|        1800|
|        11|         150|
|        12|         300|
|        13|         600|
|        14|        1800|
|        15|         450|
|        16|         700|
|        17|        1200|
|        18|        1350|
|        19|         150|
|        20|        1800|
+----------+------------+
only showing top 20 rows



### For each city, find the number of bookings and the average star rating.

In [38]:
spark.sql(
    """
    SELECT
        h.city,
        COUNT(b.booking_id) AS booking_count,
        AVG(h.star_rating) AS avg_star_rating
    FROM hotels h
    LEFT JOIN bookings b
    USING (hotel_id)
    GROUP BY 1
    """
).show()

+-----------------+-------------+---------------+
|             city|booking_count|avg_star_rating|
+-----------------+-------------+---------------+
|       Millertown|           13|            5.0|
|   South Leeshire|           11|            4.0|
|      Hensleyland|            6|            5.0|
|       Bryantfort|            5|            3.0|
|   Lake Markville|            9|            5.0|
|   Manningborough|            5|            5.0|
|   New Keithshire|           14|            3.0|
|      Weavermouth|           13|            4.0|
|      Andreashire|            6|            3.0|
|Port Samanthaside|            8|            4.0|
+-----------------+-------------+---------------+



In [40]:
joined_df = hotels_df.join(
    bookings_df,
    on="hotel_id",
    how="left"
)

joined_df.groupBy("city") \
    .agg(
        F.count(F.col("booking_id")).alias("booking_count"),
        F.avg(F.col("star_rating")).alias("avg_star_rating")
    ) \
    .show()

+-----------------+-------------+---------------+
|             city|booking_count|avg_star_rating|
+-----------------+-------------+---------------+
|       Millertown|           13|            5.0|
|   South Leeshire|           11|            4.0|
|      Hensleyland|            6|            5.0|
|       Bryantfort|            5|            3.0|
|   Lake Markville|            9|            5.0|
|   Manningborough|            5|            5.0|
|   New Keithshire|           14|            3.0|
|      Weavermouth|           13|            4.0|
|      Andreashire|            6|            3.0|
|Port Samanthaside|            8|            4.0|
+-----------------+-------------+---------------+



### Use window functions to rank hotels by revenue in each city.

In [43]:
spark.sql(
    """
    WITH revenue AS (
        SELECT
            b.hotel_id,
            h.city,
            h.hotel_name,
            SUM(b.num_nights * b.room_rate) AS total_revenue
        FROM bookings b
        LEFT JOIN hotels h
        USING (hotel_id)
        GROUP BY 1, 2, 3
    )
    
    SELECT
        city,
        hotel_id,
        hotel_name,
        total_revenue,
        DENSE_RANK() OVER (PARTITION BY city ORDER BY total_revenue DESC) AS rnk
    FROM revenue
    """
).show()

+-----------------+--------+--------------------+-------------+---+
|             city|hotel_id|          hotel_name|total_revenue|rnk|
+-----------------+--------+--------------------+-------------+---+
|      Andreashire|       2|         Smith Group|         3250|  1|
|       Bryantfort|       9|        Knight Group|         5050|  1|
|      Hensleyland|       3|        Roach-Jarvis|         8350|  1|
|   Lake Markville|       7|          Chavez PLC|         8800|  1|
|   Manningborough|       1|        Lin-Williams|         7650|  1|
|       Millertown|       5|Salazar, Porter a...|        14800|  1|
|   New Keithshire|       4|         Huffman Inc|         9750|  1|
|Port Samanthaside|      10|           Lopez Ltd|         7950|  1|
|   South Leeshire|       6|      Clark and Sons|         8950|  1|
|      Weavermouth|       8|Davis, Crane and ...|        10300|  1|
+-----------------+--------+--------------------+-------------+---+



In [45]:
joined_df = bookings_df.join(
    hotels_df,
    on="hotel_id",
    how="left"
)

grouped_df = joined_df.groupBy("hotel_id", "city", "hotel_name") \
    .agg(F.sum(F.col("num_nights") * F.col("room_rate")).alias("total_revenue"))

window_spec = Window.partitionBy(F.col("city")).orderBy(F.col("total_revenue").desc())

grouped_df.withColumn(
    "rnk",
    F.dense_rank().over(window_spec)
).select(
    "city",
    "hotel_id",
    "hotel_name",
    "total_revenue",
    "rnk"
).show()

+-----------------+--------+--------------------+-------------+---+
|             city|hotel_id|          hotel_name|total_revenue|rnk|
+-----------------+--------+--------------------+-------------+---+
|      Andreashire|       2|         Smith Group|         3250|  1|
|       Bryantfort|       9|        Knight Group|         5050|  1|
|      Hensleyland|       3|        Roach-Jarvis|         8350|  1|
|   Lake Markville|       7|          Chavez PLC|         8800|  1|
|   Manningborough|       1|        Lin-Williams|         7650|  1|
|       Millertown|       5|Salazar, Porter a...|        14800|  1|
|   New Keithshire|       4|         Huffman Inc|         9750|  1|
|Port Samanthaside|      10|           Lopez Ltd|         7950|  1|
|   South Leeshire|       6|      Clark and Sons|         8950|  1|
|      Weavermouth|       8|Davis, Crane and ...|        10300|  1|
+-----------------+--------+--------------------+-------------+---+



### Using ROW_NUMBER, find the most recent booking per guest.

In [50]:
spark.sql(
    """
    WITH chk_date AS (
        SELECT
            b.guest_id,
            g.guest_name,
            b.check_in_date,
            ROW_NUMBER () OVER (PARTITION BY b.guest_id ORDER BY b.check_in_date DESC) AS chk_in
        FROM bookings b
        JOIN guests g
        USING (guest_id)
    )
    
    SELECT
        guest_id,
        guest_name,
        check_in_date
    FROM chk_date
    WHERE chk_in = 1
    """
).show()

+--------+------------------+-------------+
|guest_id|        guest_name|check_in_date|
+--------+------------------+-------------+
|       1|     Sandra Turner|   2025-02-17|
|       3|      Jamie Murphy|   2025-01-08|
|       4|  Ashley Carpenter|   2024-01-09|
|       5| Lindsay Blake DDS|   2024-11-21|
|       6|        Michael Le|   2024-08-02|
|       7|      Diane Howard|   2024-05-16|
|       8|   Jennifer Grimes|   2025-01-07|
|       9|     Brett Shields|   2025-03-25|
|      10|       James Green|   2024-05-23|
|      11|        Gary Cantu|   2023-08-25|
|      12| Christopher Smith|   2023-11-05|
|      13|Christopher Murphy|   2024-12-06|
|      14|      Dennis Allen|   2024-05-17|
|      15|    Amanda Daniels|   2024-02-02|
|      16|      Daniel Wells|   2025-04-03|
|      18|       Sandra Wall|   2024-06-03|
|      19|        Lisa Jones|   2025-03-09|
|      21|       Emily Moore|   2024-01-18|
|      22|      Mary Jimenez|   2025-04-09|
|      23|    Mary Alexander|   

In [51]:
joined_df = bookings_df.join(
    guests_df,
    on="guest_id",
    how="inner"
)

window_spec = Window.partitionBy(F.col("guest_id")).orderBy(F.col("check_in_date").desc())

added_col = joined_df.withColumn(
    "chk_in",
    F.row_number().over(window_spec)
)

added_col.filter(F.col("chk_in") == 1) \
    .select(
        "guest_id",
        "guest_name",
        "check_in_date"
    ).show()

+--------+------------------+-------------+
|guest_id|        guest_name|check_in_date|
+--------+------------------+-------------+
|       1|     Sandra Turner|   2025-02-17|
|       3|      Jamie Murphy|   2025-01-08|
|       4|  Ashley Carpenter|   2024-01-09|
|       5| Lindsay Blake DDS|   2024-11-21|
|       6|        Michael Le|   2024-08-02|
|       7|      Diane Howard|   2024-05-16|
|       8|   Jennifer Grimes|   2025-01-07|
|       9|     Brett Shields|   2025-03-25|
|      10|       James Green|   2024-05-23|
|      11|        Gary Cantu|   2023-08-25|
|      12| Christopher Smith|   2023-11-05|
|      13|Christopher Murphy|   2024-12-06|
|      14|      Dennis Allen|   2024-05-17|
|      15|    Amanda Daniels|   2024-02-02|
|      16|      Daniel Wells|   2025-04-03|
|      18|       Sandra Wall|   2024-06-03|
|      19|        Lisa Jones|   2025-03-09|
|      21|       Emily Moore|   2024-01-18|
|      22|      Mary Jimenez|   2025-04-09|
|      23|    Mary Alexander|   